<a href="https://colab.research.google.com/github/veldahung/house_price_taiwan/blob/main/houseprice_taiwan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
import sklearn
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from xgboost import XGBRegressor
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
import math
import sklearn.model_selection as ms
import sklearn.metrics as sklm
import pandas as pd
from sklearn.model_selection import KFold
from sklearn import preprocessing
np.random.seed(3)

In [ ]:
url_train = 'https://raw.githubusercontent.com/veldahung/house_price/main/training_data.csv'
train = pd.read_csv(url_train)
print(train.head())
print(train.shape)

url_test = 'https://raw.githubusercontent.com/veldahung/house_price/main/public_dataset.csv'
test= pd.read_csv(url_test)
print(test.head())
print(test.shape)

url_test_private = 'https://raw.githubusercontent.com/veldahung/house_price/main/private_dataset.csv'
test_new= pd.read_csv(url_test_private)
print(test_new.head())
print(test_new.shape)

     ID   縣市 鄉鎮市區      路名      土地面積  使用分區  移轉層次  總樓層數  主要用途    主要建材  ...  \
0  TR-1  台北市  大安區  敦化南路二段 -0.256716  None    11    11   住家用  鋼筋混凝土造  ...   
1  TR-2  台北市  萬華區     水源路  0.100134  None     7    12   住家用  鋼筋混凝土造  ...   
2  TR-3  高雄市  鳳山區     北忠街  0.181921  None    10    15  集合住宅      其他  ...   
3  TR-4  新北市  新莊區     福前街  0.085594  None     9    14  集合住宅  鋼筋混凝土造  ...   
4  TR-5  新北市  板橋區   文化路一段 -0.938116  None    41    43   住家用     鋼骨造  ...   

       建物面積      車位面積  車位個數     橫坐標      縱坐標   備註     主建物面積      陽台面積  \
0 -0.174154 -0.819326   0.0  305266  2768378  NaN  0.393926  0.183700   
1  0.314204 -0.819326   0.0  300677  2767990  NaN -0.316131  0.608577   
2  0.423366  0.161624   1.0  184815  2504666  NaN -0.098871 -0.360620   
3  0.164249  0.524653   1.0  296653  2772355  NaN -0.071147  0.315088   
4  0.985839  0.532377   1.0  297377  2768472  NaN  0.791954  1.719400   

     附屬建物面積        單價  
0 -0.438452  4.627714  
1 -0.438452  1.887258  
2  1.525881  1.489072  
3  0.231

In [ ]:
puby=[]
puby=test["ID"]
puby=puby.append(test_new["ID"])

<ipython-input-4-2619cc3cadc0>:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  puby=puby.append(test_new["ID"])


In [ ]:
data_all = pd.concat((train, test, test_new), sort=False).reset_index(drop=True)

In [ ]:
# numeric feature describe
num_data=data_all.select_dtypes(['int64','float64'])
describe_num=data_all.describe().transpose()
print (describe_num)
print(num_data.columns)

          count          mean           std           min           25%  \
土地面積    23502.0  8.102521e-17      1.000021 -1.699811e+00 -6.517617e-01   
移轉層次    23502.0  7.352225e+00      4.994471  2.000000e+00  4.000000e+00   
總樓層數    23502.0  1.248434e+01      6.673036  2.000000e+00  7.000000e+00   
屋齡      23502.0  2.130552e+01     14.425636  0.000000e+00  8.083333e+00   
建物面積    23502.0 -1.402825e-16      1.000021 -1.909259e+00 -6.176972e-01   
車位面積    23502.0 -1.100492e-16      1.000021 -8.193257e-01 -8.193257e-01   
車位個數    23502.0  6.285848e-01      0.668707  0.000000e+00  0.000000e+00   
橫坐標     23502.0  2.683703e+05  49941.266876 -2.339100e+04  2.381920e+05   
縱坐標     23502.0  2.719582e+06  92136.788959  2.493809e+06  2.729189e+06   
主建物面積   23502.0 -3.640088e-16      1.000021 -2.098629e+00 -6.097724e-01   
陽台面積    23502.0  2.418663e-17      1.000021 -1.642077e+00 -6.285149e-01   
附屬建物面積  23502.0 -7.497855e-17      1.000021 -4.384519e-01 -4.384519e-01   
單價      11751.0  1.993343

In [ ]:
# KDE-all

'''
for i in list(num_data.columns):
    sns.distplot(num_data[i].dropna()).set_title(i)
    plt.show()
'''


'\nfor i in list(num_data.columns):\n    sns.distplot(num_data[i].dropna()).set_title(i)\n    plt.show()\n'

In [ ]:
# correlation heatmap
num_train=train.select_dtypes(['int64','float64'])
num_corr=num_train.corr()
#print(num_corr)
print(num_corr.sort_values(by=['單價'], ascending=False).head(1))
'''
fig,ax=plt.subplots(figsize=(15,1))
sns.heatmap(num_corr.sort_values(by=['單價'], ascending=False).head(1), cmap='Reds')
plt.title("Correlation Matrix", weight='bold', fontsize=18)
plt.xticks(weight='bold')
plt.yticks(weight='bold', color='dodgerblue', rotation=0)
'''


        土地面積      移轉層次      總樓層數        屋齡      建物面積      車位面積      車位個數  \
單價  0.067088  0.054038  0.037874  0.049196  0.061327  0.099986  0.013064   

         橫坐標       縱坐標    主建物面積      陽台面積    附屬建物面積   單價  
單價  0.455633  0.394402  0.05615  0.009866  0.064905  1.0  


'\nfig,ax=plt.subplots(figsize=(15,1))\nsns.heatmap(num_corr.sort_values(by=[\'單價\'], ascending=False).head(1), cmap=\'Reds\')\nplt.title("Correlation Matrix", weight=\'bold\', fontsize=18)\nplt.xticks(weight=\'bold\')\nplt.yticks(weight=\'bold\', color=\'dodgerblue\', rotation=0)\n'

In [ ]:
# 特徵描述-類別
cat_data=data_all.select_dtypes(['object'])
print(cat_data)
'''
for i in list(cat_data.columns):
    fig,ax=plt.subplots(1,1)
    ax.hist(cat_data[i].dropna())
    ax.set_title(i,fontsize=12,color='Black')
    plt.show()
'''

            ID   縣市 鄉鎮市區      路名  使用分區   主要用途    主要建材             建物型態   備註
0         TR-1  台北市  大安區  敦化南路二段  None    住家用  鋼筋混凝土造  住宅大樓(11層含以上有電梯)  NaN
1         TR-2  台北市  萬華區     水源路  None    住家用  鋼筋混凝土造  住宅大樓(11層含以上有電梯)  NaN
2         TR-3  高雄市  鳳山區     北忠街  None   集合住宅      其他  住宅大樓(11層含以上有電梯)  NaN
3         TR-4  新北市  新莊區     福前街  None   集合住宅  鋼筋混凝土造  住宅大樓(11層含以上有電梯)  NaN
4         TR-5  新北市  板橋區   文化路一段  None    住家用     鋼骨造  住宅大樓(11層含以上有電梯)  NaN
...        ...  ...  ...     ...   ...    ...     ...              ...  ...
23497  PR-5871  高雄市  鳳山區     和德街  None   集合住宅  鋼筋混凝土造  住宅大樓(11層含以上有電梯)  NaN
23498  PR-5872  台北市  南港區     玉成街  None  一般事務所      其他  住宅大樓(11層含以上有電梯)  NaN
23499  PR-5873  新北市  林口區  文化二路一段  None    住家用  鋼筋混凝土造  住宅大樓(11層含以上有電梯)  NaN
23500  PR-5874  高雄市  鳳山區    文龍東路  None   集合住宅  鋼筋混凝土造  住宅大樓(11層含以上有電梯)  NaN
23501  PR-5875  新竹縣  竹北市   十興路一段  None    住家用  鋼筋混凝土造  住宅大樓(11層含以上有電梯)  NaN

[23502 rows x 9 columns]


"\nfor i in list(cat_data.columns):\n    fig,ax=plt.subplots(1,1)\n    ax.hist(cat_data[i].dropna())\n    ax.set_title(i,fontsize=12,color='Black')\n    plt.show()\n"

In [ ]:
missing_columns=data_all.isnull().mean().sort_values(ascending=False)
missing_columns=missing_columns[missing_columns!=0].to_frame().reset_index()
print(missing_columns)

  index         0
0    備註  0.992256
1    單價  0.500000


In [ ]:

# 檢查連續變項跟依變數的分布情形
data_remove_outlier=data_all[:11751]
'''
print(num_train)
for c in num_train:
    sns.scatterplot(x=c, y="單價", data=data_remove_outlier).set_title(c,fontsize=12,color='Black')
    plt.show()
'''


'\nprint(num_train)\nfor c in num_train:\n    sns.scatterplot(x=c, y="單價", data=data_remove_outlier).set_title(c,fontsize=12,color=\'Black\')\n    plt.show()\n'

In [ ]:

outlier_columns=['土地面積','移轉層次','總樓層數','建物面積','車位面積', '主建物面積','陽台面積','附屬建物面積']
outlier_threshold=[15,40,60,10,8,12,12,30]
for c,n in zip(outlier_columns,outlier_threshold):
    data_remove_outlier=data_remove_outlier[data_remove_outlier[c]<n]
print(data_remove_outlier.shape)
data_all = pd.concat((data_remove_outlier, data_all[11751:]), sort=False).reset_index(drop=True)
print(data_all.shape)


(11738, 22)
(23489, 22)


In [ ]:
# 遺漏值檢查
missing_columns=data_all.isnull().mean().sort_values(ascending=False)
missing_columns=missing_columns[missing_columns!=0].to_frame().reset_index()
print(missing_columns)

'''
# 遺漏率繪圖
fig,ax=plt.subplots(figsize=(7,7))
sns.barplot(x=0,y='index',data=missing_columns)
'''

  index         0
0    備註  0.992252
1    單價  0.500277


"\n# 遺漏率繪圖\nfig,ax=plt.subplots(figsize=(7,7))\nsns.barplot(x=0,y='index',data=missing_columns)\n"

In [ ]:
# 移除遺漏值太多或分布太奇怪的欄位
data_all=data_all.drop(columns=['ID'],axis=1)
data_all=data_all.drop(columns=['備註'],axis=1)
missing_num=['單價']
#missing_num=['單價','陽台面積','主建物面積','土地面積','建物面積']
# 數值補0
for i in missing_num:
    data_all[i]=data_all[i].fillna(0)

In [ ]:
# 遺漏值檢查
missing_columns=data_all.isnull().mean().sort_values(ascending=False)
missing_columns=missing_columns[missing_columns!=0].to_frame().reset_index()
print(missing_columns)

Empty DataFrame
Columns: [index, 0]
Index: []


In [ ]:
'''
#面積相關欄位_對數轉換
for i in list(['土地面積','移轉層次','總樓層數','建物面積','車位面積','主建物面積','陽台面積','附屬建物面積']):
    data_all[i]=(np.log1p(data_all[i].dropna()))
'''

"\n#面積相關欄位_對數轉換\nfor i in list(['土地面積','移轉層次','總樓層數','建物面積','車位面積','主建物面積','陽台面積','附屬建物面積']):\n    data_all[i]=(np.log1p(data_all[i].dropna()))\n"

In [ ]:
'''
# 遺漏值檢查
missing_columns=data_all.isnull().mean().sort_values(ascending=False)
missing_columns=missing_columns[missing_columns!=0].to_frame().reset_index()
print(missing_columns)
'''

'\n# 遺漏值檢查\nmissing_columns=data_all.isnull().mean().sort_values(ascending=False)\nmissing_columns=missing_columns[missing_columns!=0].to_frame().reset_index()\nprint(missing_columns)\n'

In [ ]:
'''
missing_num=['陽台面積','主建物面積','土地面積','建物面積']
# 數值補0
for i in missing_num:
    data_all[i]=data_all[i].fillna(0)
'''

"\nmissing_num=['陽台面積','主建物面積','土地面積','建物面積']\n# 數值補0\nfor i in missing_num:\n    data_all[i]=data_all[i].fillna(0)\n"

In [ ]:

# 遺漏值檢查
missing_columns=data_all.isnull().mean().sort_values(ascending=False)
missing_columns=missing_columns[missing_columns!=0].to_frame().reset_index()
print(missing_columns)


Empty DataFrame
Columns: [index, 0]
Index: []


In [ ]:
#data_all['地址']=data_all['縣市']+data_all['鄉鎮市區']+data_all['路名']

In [ ]:
'''
data=data_all[:11738]
# Define categorical features to encode
categorical_cols = ['縣市', '鄉鎮市區','路名','使用分區','主要用途','主要建材','建物型態' ]
#categorical_cols = ['地址','使用分區','主要用途','主要建材','建物型態' ]
#categorical_cols = ['使用分區','主要用途','主要建材','建物型態' ]

#categorical_cols = ['路名' ]

# Create K-fold cross-validation
kf = KFold(n_splits=27, shuffle=True, random_state=42)

# Initialize the TargetEncoder with smoothing (regularization)
encoder = TargetEncoder(cols=categorical_cols, smoothing=0.4)



# Perform target encoding within each fold
for train_index, val_index in kf.split(data):
    X_train, X_val = data.iloc[train_index].drop(columns=['單價'],axis=1), data.iloc[val_index].drop(columns=['單價'],axis=1)
    y_train, y_val = data.iloc[train_index][['單價']], data.iloc[val_index][['單價']]

    encoder.fit(X_train, y_train)
    X_train_encoded = encoder.transform(X_train)
    X_val_encoded = encoder.transform(X_val)

    # Use X_train_encoded and X_val_encoded in your modeling pipeline


'''
'''
# Initialize the TargetEncoder with smoothing (regularization)
encoder = TargetEncoder(cols=categorical_cols, smoothing=0.37)


X=data.drop(columns=['單價'],axis=1)
y=data[['單價']]
encoder.fit(X, y)
X_encoded=encoder.transform(X)

for train_index, val_index in kf.split(data):
    X_train_encoded, X_val_encoded = X_encoded.iloc[train_index], X_encoded.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
'''

#X_train_encoded=X_train_encoded.drop("陽台面積",axis = 1)
#X_val_encoded=X_val_encoded.drop("陽台面積",axis = 1)

#X_train_encoded=X_train_encoded.drop("主建物面積",axis = 1)
#X_train_encoded=X_train_encoded.drop("主要建材",axis = 1)
#X_train_encoded=X_train_encoded.drop("路名",axis = 1)
#X_train_encoded=X_train_encoded.drop("鄉鎮市區",axis = 1)
#X_train_encoded=X_train_encoded.drop("縣市",axis = 1)
#X_val_encoded=X_val_encoded.drop("主建物面積",axis = 1)
#X_val_encoded=X_val_encoded.drop("主要建材",axis = 1)
#X_val_encoded=X_val_encoded.drop("路名",axis = 1)
#X_val_encoded=X_val_encoded.drop("鄉鎮市區",axis= 1)
#X_val_encoded=X_val_encoded.drop("縣市",axis= 1)
'''
print(X_train_encoded.shape)
print(X_val_encoded.shape)
'''

'\nprint(X_train_encoded.shape)\nprint(X_val_encoded.shape)\n'

In [ ]:

data=data_all[:11738]
X=data.drop(columns=['單價'],axis=1)
y=data[['單價']]
# Define categorical features to encode
categorical_cols = ['縣市', '鄉鎮市區','路名','使用分區','主要用途','主要建材','建物型態' ]

encoder = TargetEncoder(cols=categorical_cols, smoothing=0.37)


encoder.fit(X, y)
X_encoded=encoder.transform(X)


#X_train_encoded, X_val_encoded, y_train, y_val = train_test_split(X_encoded, y, test_size=0.2,random_state=30)

missing_columns=X_encoded.isnull().mean().sort_values(ascending=False)
missing_columns=missing_columns[missing_columns!=0].to_frame().reset_index()
print(missing_columns)


Empty DataFrame
Columns: [index, 0]
Index: []


In [ ]:
#X_train_encoded.head()

In [ ]:
data_test=data_all[11738:]
print(data_test.shape)
X_test=data_test.drop(columns=['單價'],axis=1)
y_predict=data_test[['單價']]
# Assuming you already have the encoder object from the training step
X_test_encoded = encoder.transform(X_test)

#X_test_encoded=X_test_encoded.drop("陽台面積",axis = 1)
#X_test_encoded=X_test_encoded.drop("主建物面積",axis = 1)
#X_test_encoded=X_test_encoded.drop("主要建材",axis = 1)
#X_test_encoded=X_test_encoded.drop("路名",axis = 1)
#X_test_encoded=X_test_encoded.drop("鄉鎮市區",axis = 1)
#X_test_encoded=X_test_encoded.drop("縣市",axis = 1)

X_test_encoded.head()
print(X_test_encoded.shape)

(11751, 20)
(11751, 19)


In [ ]:
'''
print(X_train_encoded.shape)
print(X_val_encoded.shape)
'''
#X= pd.concat((X_train_encoded, X_val_encoded,X_test_encoded), sort=False).reset_index(drop=True)
X= pd.concat((X_encoded,X_test_encoded), sort=False).reset_index(drop=True)
'''
Y=pd.concat((y_train, y_val), sort=False).reset_index(drop=True)
print(Y)
missing_columns=X.isnull().mean().sort_values(ascending=False)
missing_columns=missing_columns[missing_columns!=0].to_frame().reset_index()
print(missing_columns)
'''

'\nY=pd.concat((y_train, y_val), sort=False).reset_index(drop=True)\nprint(Y)\nmissing_columns=X.isnull().mean().sort_values(ascending=False)\nmissing_columns=missing_columns[missing_columns!=0].to_frame().reset_index()\nprint(missing_columns)\n'

In [ ]:
'''
missing_num=['陽台面積','主建物面積','土地面積','建物面積']
# 數值補0
for i in missing_num:
    X[i]=X[i].fillna(0)
'''

"\nmissing_num=['陽台面積','主建物面積','土地面積','建物面積']\n# 數值補0\nfor i in missing_num:\n    X[i]=X[i].fillna(0)\n"

In [ ]:

X=preprocessing.normalize(X,axis=0)
#Y=preprocessing.normalize(Y)
Columns=['縣市', '鄉鎮市區', '路名', '土地面積', '使用分區', '移轉層次', '總樓層數', '主要用途', '主要建材', '建物型態', '屋齡', '建物面積', '車位面積', '車位個數', '橫坐標', '縱坐標', '主建物面積', '陽台面積', '附屬建物面積']
X = pd.DataFrame(X,columns=Columns)
print(X)


             縣市      鄉鎮市區        路名      土地面積      使用分區      移轉層次      總樓層數  \
0      0.010325  0.013581  0.006383 -0.001698  0.006482  0.008090  0.005078   
1      0.010325  0.007569  0.006383  0.000662  0.006482  0.005148  0.005539   
2      0.003805  0.003268  0.006383  0.001204  0.006482  0.007355  0.006924   
3      0.006010  0.005982  0.006383  0.000566  0.006482  0.006619  0.006462   
4      0.010325  0.007569  0.006383  0.001723  0.006482  0.001471  0.005539   
...         ...       ...       ...       ...       ...       ...       ...   
23484  0.003805  0.003268  0.006383 -0.001080  0.006482  0.018387  0.011540   
23485  0.010325  0.009973  0.006383  0.007401  0.006482  0.004413  0.007847   
23486  0.006010  0.005428  0.005392 -0.000352  0.006482  0.010296  0.006462   
23487  0.003805  0.003268  0.006383 -0.000594  0.006482  0.001471  0.006924   
23488  0.005593  0.006079  0.006383  0.009761  0.006482  0.007355  0.006462   

           主要用途      主要建材      建物型態        屋齡      

In [ ]:

# 遺漏值檢查
missing_columns=X.isnull().mean().sort_values(ascending=False)
missing_columns=missing_columns[missing_columns!=0].to_frame().reset_index()
print(missing_columns)


Empty DataFrame
Columns: [index, 0]
Index: []


In [ ]:
'''
y_predict_test=xgbc.predict(X_test_encoded)
print(y_predict_test[:10])
'''

'\ny_predict_test=xgbc.predict(X_test_encoded)\nprint(y_predict_test[:10])\n'

In [ ]:

x=X[:11738]
#X_train_encoded, X_val_encoded, y_train, y_val = train_test_split(x, y, test_size=0.2,random_state=30)
X_train_encoded=x
y_train=y
#X_train_encoded=X[:11304]
#X_val_encoded=X[11304:11738]
#print(X_val_encoded.shape)


In [ ]:

X_test_encoded=X[11738:]
print(X_test_encoded.shape)


(11751, 19)


In [ ]:
'''
xgbc=XGBRegressor(max_depth=10,learning_rate=0.05,n_estimators=503,colsample_bytree=0.87,reg_alpha= 0.55,reg_lambda=0.2)
#xgbc=XGBRegressor(learning_rate= 0.1, n_estimators= 800, max_depth= 7, min_child_weight= 2, seed= 0,
                    #subsample= 0.8, colsample_bytree= 0.7, gamma= 0.1, reg_alpha= 0.05, reg_lambda= 2)
'''

'\nxgbc=XGBRegressor(max_depth=10,learning_rate=0.05,n_estimators=503,colsample_bytree=0.87,reg_alpha= 0.55,reg_lambda=0.2)\n#xgbc=XGBRegressor(learning_rate= 0.1, n_estimators= 800, max_depth= 7, min_child_weight= 2, seed= 0,\n                    #subsample= 0.8, colsample_bytree= 0.7, gamma= 0.1, reg_alpha= 0.05, reg_lambda= 2)\n'

In [ ]:
'''
yp=xgbc.fit(X_train_encoded,y_train)
y_train_predict=xgbc.predict(X_train_encoded)
'''

'\nyp=xgbc.fit(X_train_encoded,y_train)\ny_train_predict=xgbc.predict(X_train_encoded)\n'

In [ ]:
'''
print('accuracy:',xgbc.score(X_train_encoded,y_train))

print(mse(y_train_predict,y_train))
'''

"\nprint('accuracy:',xgbc.score(X_train_encoded,y_train))\n\nprint(mse(y_train_predict,y_train))\n"

In [ ]:
'''
y_val_predict=xgbc.predict(X_val_encoded)
'''

'\ny_val_predict=xgbc.predict(X_val_encoded)\n'

In [ ]:
'''
print('accuracy:',xgbc.score(X_val_encoded,y_val))
print(mse(y_val_predict,y_val))
continuous_data = y_val

array_data = np.array(continuous_data)

continuous_data = y_val_predict

y_pre = np.array(continuous_data)

#print(array_data)
#print(y_pre)
length = y_pre.shape[0]
total=0
for i in range(length):
    total+=abs(((array_data[i]-y_pre[i])/array_data[i]))
total=total/length
print(total)
'''

"\nprint('accuracy:',xgbc.score(X_val_encoded,y_val))\nprint(mse(y_val_predict,y_val))\ncontinuous_data = y_val\n\narray_data = np.array(continuous_data)\n\ncontinuous_data = y_val_predict\n\ny_pre = np.array(continuous_data)\n\n#print(array_data)\n#print(y_pre)\nlength = y_pre.shape[0]\ntotal=0\nfor i in range(length):\n    total+=abs(((array_data[i]-y_pre[i])/array_data[i]))\ntotal=total/length\nprint(total)\n"

In [ ]:
#y_test_predict=xgbc.predict(X_test_encoded)

In [ ]:
'''
presult = np.vstack((puby, y_test_predict))
print(presult.T)


df = pd.DataFrame(presult.T,columns=['ID','predicted_price'])
df.to_csv("myfile.csv",index=False)

from google.colab import files
#files.download("myfile.csv")
'''

'\npresult = np.vstack((puby, y_test_predict))\nprint(presult.T)\n\n\ndf = pd.DataFrame(presult.T,columns=[\'ID\',\'predicted_price\'])\ndf.to_csv("myfile.csv",index=False)\n\nfrom google.colab import files\n#files.download("myfile.csv")\n'

In [ ]:
from sklearn.linear_model import Ridge
# The alpha used by Python's ridge should be the lambda in Hull's book times the number of observations
alphas=[0,0.0001,0.0002]
msev=[]
mset=[]
msed=[]
for alpha in alphas:
    ridge=Ridge(alpha=alpha)
    ridge.fit(X_train_encoded,y_train)
    pred_t=ridge.predict(X_train_encoded)
    mset.append(mse(y_train,pred_t))
    #pred=ridge.predict(X_val_encoded)
    #msev.append(mse(y_val,pred))
    print(mse(y_train,pred_t))
    #print(mse(y_val,pred))
    #print(mse(y_train,pred_t)-mse(y_val,pred))
    #msed.append(mse(y_val,pred)-mse(y_train,pred_t))

'''
dr = {"training": mset, "validation": msev, "difference": msed}
dfr = pd.DataFrame(data=dr, index=alphas)
display(dfr)
'''
'''
ridge=Ridge(alpha=0.00001)
ridge.fit(X_train_encoded,y_train)
ridge_v=ridge.predict(X_val_encoded)
'''

0.2185687087374451
0.21881284075830418
0.21901592538436926


'\nridge=Ridge(alpha=0.00001)\nridge.fit(X_train_encoded,y_train)\nridge_v=ridge.predict(X_val_encoded)\n'

In [ ]:
'''
continuous_data = y_val

array_data = np.array(continuous_data)

continuous_data = ridge_v

y_pre = np.array(continuous_data)

#print(array_data)
#print(y_pre)
length = y_pre.shape[0]
total=0
for i in range(length):
    total+=abs(((array_data[i]-y_pre[i])/array_data[i]))
total=total/length
print(total)
'''

'\ncontinuous_data = y_val\n\narray_data = np.array(continuous_data)\n\ncontinuous_data = ridge_v\n\ny_pre = np.array(continuous_data)\n\n#print(array_data)\n#print(y_pre)\nlength = y_pre.shape[0]\ntotal=0\nfor i in range(length):\n    total+=abs(((array_data[i]-y_pre[i])/array_data[i]))\ntotal=total/length\nprint(total)\n'

In [ ]:

from sklearn.linear_model import LassoCV
# set cross-validation alpha
alpha=[0.0001,0.001,0.01,0.1,1,10,100]
# find the best alpha and build model
Lasso = LassoCV(cv=100, alphas=alpha)
Lasso_fit=Lasso.fit(X_train_encoded,y_train)
y_lasso_train=Lasso_fit.predict(X_train_encoded)
#y_lasso_val=Lasso_fit.predict(X_val_encoded)
y_lasso_test=Lasso_fit.predict(X_test_encoded)
# validation( train data and validate data)
print('RMSE_train_Lasso = ' + str(math.sqrt(sklm.mean_squared_error(y_train, y_lasso_train))))
#print('RMSE_test_Lasso = ' + str(math.sqrt(sklm.mean_squared_error(y_val, y_lasso_val))))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE_train_Lasso = 0.48584795487917887


In [ ]:
'''
from sklearn.linear_model import ElasticNetCV
# set cross-validation alpha and l1ratio
alpha=[0.0001,0.001,0.01,0.1,1,10,100]
l1ratio = [0.1, 0.5, 0.9, 0.95, 0.99, 1]
# find the best alpha/l1ratio and build model
elastic_cv = ElasticNetCV(cv=5, max_iter=1, alphas=alpha, l1_ratio=l1ratio)
elastic_fit = elastic_cv.fit(X_train_encoded, y_train)
y_el_train=elastic_fit.predict(X_train_encoded)
y_el_val=elastic_fit.predict(X_val_encoded)
y_el_test=elastic_fit.predict(X_test_encoded)
# validation( train data and validate data)
print('RMSE_train_ElasticNet = ' + str(math.sqrt(sklm.mean_squared_error(y_train, y_el_train))))
print('RMSE_test_ElasticNet = ' + str(math.sqrt(sklm.mean_squared_error(y_val, y_el_val))))
'''

"\nfrom sklearn.linear_model import ElasticNetCV\n# set cross-validation alpha and l1ratio\nalpha=[0.0001,0.001,0.01,0.1,1,10,100]\nl1ratio = [0.1, 0.5, 0.9, 0.95, 0.99, 1]\n# find the best alpha/l1ratio and build model\nelastic_cv = ElasticNetCV(cv=5, max_iter=1, alphas=alpha, l1_ratio=l1ratio)\nelastic_fit = elastic_cv.fit(X_train_encoded, y_train)\ny_el_train=elastic_fit.predict(X_train_encoded)\ny_el_val=elastic_fit.predict(X_val_encoded)\ny_el_test=elastic_fit.predict(X_test_encoded)\n# validation( train data and validate data)\nprint('RMSE_train_ElasticNet = ' + str(math.sqrt(sklm.mean_squared_error(y_train, y_el_train))))\nprint('RMSE_test_ElasticNet = ' + str(math.sqrt(sklm.mean_squared_error(y_val, y_el_val))))\n"

In [ ]:
'''
from lightgbm import LGBMRegressor
estimator=LGBMRegressor()
parameter_grid={
        'max_depth':range(2,5,1),
        'learning_rate':np.linspace(0.001,1,20),
        'feature_fraction':np.linspace(0.5,0.99,20),
        'bagging_fraction':np.linspace(0.1,0.99,20),
        'bagging_frequency':range(5,10,1),
        'num_leaves':range(100,200,5),
        'min_data_in_leaf':range(50,200,10),
        'n_estimators':range(100,5000,100)}
grid = RandomizedSearchCV(estimator,parameter_grid,cv = 3,scoring = 'neg_root_mean_squared_error',n_iter=500,n_jobs = -1)
grid.fit(X_train_encoded, y_train)
# print best parameter combination
print (grid.best_params_)
'''

"\nfrom lightgbm import LGBMRegressor\nestimator=LGBMRegressor()\nparameter_grid={\n        'max_depth':range(2,5,1),\n        'learning_rate':np.linspace(0.001,1,20),\n        'feature_fraction':np.linspace(0.5,0.99,20),\n        'bagging_fraction':np.linspace(0.1,0.99,20),\n        'bagging_frequency':range(5,10,1),\n        'num_leaves':range(100,200,5),\n        'min_data_in_leaf':range(50,200,10),\n        'n_estimators':range(100,5000,100)}\ngrid = RandomizedSearchCV(estimator,parameter_grid,cv = 3,scoring = 'neg_root_mean_squared_error',n_iter=500,n_jobs = -1)\ngrid.fit(X_train_encoded, y_train)\n# print best parameter combination\nprint (grid.best_params_)\n"

In [ ]:
from lightgbm import LGBMRegressor

lightgbm = LGBMRegressor(objective='regression',
                         num_leaves=10,
                         min_data_in_leaf= 20,
                         min_sum_hessian_in_leaf = 11,
                         reg_alpha =0.55,
                         reg_lambda =0.2,
                         learning_rate=0.05,
                         n_estimators=1000,
                         max_bin=70,
                         metric="rmse",
                         max_depth =10,
                         bagging_fraction=0.9,
                         bagging_freq=5,
                         bagging_seed=9,
                         feature_fraction=0.9,
                         feature_fraction_seed=15,
                         verbose=-1)
lgbm_fit = lightgbm.fit(X_train_encoded, y_train)
y_lgbm_train=lgbm_fit.predict(X_train_encoded)
#y_lgbm_val=lgbm_fit.predict(X_val_encoded)
y_lgbm_test=lgbm_fit.predict(X_test_encoded)
print('RMSE_train_LGBM = ' + str(math.sqrt(sklm.mean_squared_error(y_train, y_lgbm_train))))
#print('RMSE_test_LGBM = ' + str(math.sqrt(sklm.mean_squared_error(y_val, y_lgbm_val))))

RMSE_train_LGBM = 0.22579943001655495


In [ ]:
'''
from xgboost.sklearn import XGBRegressor
estimator=XGBRegressor()
parameter_grid={
        'max_depth':range(2,5,1),
        'learning_rate':np.linspace(0.001,1,20),
        'colsample_bytree':np.linspace(0.1,0.99,20),
        'num_leaves':range(100,200,5),
        'n_estimators':range(100,5000,100)}
grid = RandomizedSearchCV(estimator,parameter_grid,cv = 5,scoring = 'neg_root_mean_squared_error',n_iter=500,n_jobs = -1)
grid.fit(X_train_encoded, y_train)
# print best parameter combination
print (grid.best_params_)
'''

"\nfrom xgboost.sklearn import XGBRegressor\nestimator=XGBRegressor()\nparameter_grid={\n        'max_depth':range(2,5,1),\n        'learning_rate':np.linspace(0.001,1,20),\n        'colsample_bytree':np.linspace(0.1,0.99,20),\n        'num_leaves':range(100,200,5),\n        'n_estimators':range(100,5000,100)}\ngrid = RandomizedSearchCV(estimator,parameter_grid,cv = 5,scoring = 'neg_root_mean_squared_error',n_iter=500,n_jobs = -1)\ngrid.fit(X_train_encoded, y_train)\n# print best parameter combination\nprint (grid.best_params_)\n"

In [ ]:
'''
xgb= XGBRegressor(base_score=0.5,
                  booster='gbtree',
                  colsample_bylevel=1,
                  colsample_bynode=1,
                  colsample_bytree=0.87,
                  gamma=0,
                  importance_type='gain',
                  learning_rate=0.05,
                  max_delta_step=0,
                  max_depth=10,
                  min_child_weight=0,
                  missing=None,
                  n_estimators=503,
                  n_jobs=1,
                  nthread=None,
                  objective='reg:squarederror',
                  random_state=0,
                  reg_alpha=0.55,
                  reg_lambda=0.2,
                  scale_pos_weight=1,
                  seed=None,
                  silent=None,
                  subsample=1,
                  verbosity=1)
'''
xgbc=XGBRegressor(max_depth=10,learning_rate=0.05,n_estimators=503,colsample_bytree=0.87,reg_alpha= 0.55,reg_lambda=0.2)
xgb_fit=xgbc.fit(X_train_encoded, y_train)
y_xgb_train=xgb_fit.predict(X_train_encoded)
#y_xgb_val=xgb_fit.predict(X_val_encoded)
y_xgb_test=xgb_fit.predict(X_test_encoded)
print('RMSE_train_LGBM = ' + str(math.sqrt(sklm.mean_squared_error(y_train, y_xgb_train))))
#print('RMSE_test_LGBM = ' + str(math.sqrt(sklm.mean_squared_error(y_val, y_xgb_val))))

RMSE_train_LGBM = 0.055347787710977876


In [ ]:

continuous_data = y_train

array_data = np.array(continuous_data)

continuous_data = y_xgb_train

y_pre = np.array(continuous_data)

#print(array_data)
#print(y_pre)
length = y_pre.shape[0]
total=0
for i in range(length):
    total+=abs(((array_data[i]-y_pre[i])/array_data[i]))
total=total/length
print(total)


[0.02417675]


In [ ]:
from sklearn.ensemble import VotingRegressor
# Build Model
#vote_mod = VotingRegressor([('Ridge', Ridge_fit), ('Lasso', Lasso_fit), ('Elastic', elastic_fit), ('lgbm', lgbm_fit),('xgb',xgb_fit)])vote= vote_mod.fit(X_train_encoded, y_train.ravel())
vote_mod = VotingRegressor([('lgbm', lgbm_fit),('xgb',xgb_fit)])
vote= vote_mod.fit(X_train_encoded, y_train)
# predict train/test y
vote_pred_train=vote.predict(X_train_encoded)
#vote_pred_val=vote.predict(X_val_encoded)
vote_pred_test=vote.predict(X_test_encoded)
# validation( train data and validate data)
print('RMSE_train_Voting = ' + str(math.sqrt(sklm.mean_squared_error(y_train, vote_pred_train))))
#print('RMSE_test_Voting = ' + str(math.sqrt(sklm.mean_squared_error(y_val, vote_pred_val))))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_voting.py:597: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE_train_Voting = 0.13395560522100414


In [ ]:
continuous_data = y_train

array_data = np.array(continuous_data)

continuous_data = vote_pred_train

y_pre = np.array(continuous_data)

#print(array_data)
#print(y_pre)
length = y_pre.shape[0]
total=0
for i in range(length):
    total+=abs(((array_data[i]-y_pre[i])/array_data[i]))
total=total/length
print(total)

[0.05476461]


In [ ]:
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
# Build Model
gbdt=GradientBoostingRegressor(learning_rate=0.05,
                               max_leaf_nodes=10,
                              n_estimators=1000)
#stregr = StackingRegressor(regressors=[Ridge_fit, Lasso_fit, elastic_fit, lgbm_fit,xgb_fit],meta_regressor=gbdt,use_features_in_secondary=True)
stregr = StackingRegressor(regressors=[lgbm_fit,xgb_fit],
                           meta_regressor=gbdt,
                           use_features_in_secondary=True)
stack_mod=stregr.fit(X_train_encoded, y_train)
# predict train/test y
stacking_pred_train=stack_mod.predict(X_train_encoded)
#stacking_pred_val=stack_mod.predict(X_val_encoded)
stacking_pred_test=stack_mod.predict(X_test_encoded)
# validation( train data and validate data)
print('RMSE_train_Stacking = ' + str(math.sqrt(sklm.mean_squared_error(y_train, stacking_pred_train))))
#print('RMSE_test_Stacking = ' + str(math.sqrt(sklm.mean_squared_error(y_val, stacking_pred_val))))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE_train_Stacking = 0.038899637750067606


In [ ]:
continuous_data = y_train

array_data = np.array(continuous_data)

continuous_data = stacking_pred_train

y_pre = np.array(continuous_data)

#print(array_data)
#print(y_pre)
length = y_pre.shape[0]
total=0
for i in range(length):
    total+=abs(((array_data[i]-y_pre[i])/array_data[i]))
total=total/length
print(total)

[0.0172583]


In [ ]:
# set weight list
weight=list(np.linspace(0.1,1,91))
# create outcome dataframe
train_mse=[]
val_mse=[]
for i in weight:
    blending_pred_train=(i*vote_pred_train)+((1-i)*stacking_pred_train)
    #blending_pred_val=(i*vote_pred_val)+((1-i)*stacking_pred_val)
    blending_pred_test=(i*vote_pred_test)+((1-i)*stacking_pred_test)
    train_mse.append(math.sqrt(sklm.mean_squared_error(y_train, blending_pred_train)))
    #val_mse.append(math.sqrt(sklm.mean_squared_error(y_val, blending_pred_val)))
#blending_output=pd.DataFrame({'weight':weight,'train_mse':train_mse,'test_mse':val_mse})
# print top 10 weight value
#print (blending_output.sort_values(by=['test_mse'],ascending=True).head(10))

blending_pred_test=(0.75*vote_pred_test)+((1-0.75)*stacking_pred_test)

In [ ]:
continuous_data = y_train

array_data = np.array(continuous_data)

continuous_data = blending_pred_train

y_pre = np.array(continuous_data)

#print(array_data)
#print(y_pre)
length = y_pre.shape[0]
total=0
for i in range(length):
    total+=abs(((array_data[i]-y_pre[i])/array_data[i]))
total=total/length
print(total)

[0.05476461]


In [ ]:
presult = np.vstack((puby, stacking_pred_test))
print(presult.T)


df = pd.DataFrame(presult.T,columns=['ID','predicted_price'])
df.to_csv("myfile.csv",index=False)

from google.colab import files
files.download("myfile.csv")

[['PU-1' 1.5804004769065139]
 ['PU-2' 1.6930247079916694]
 ['PU-3' 2.659295352919086]
 ...
 ['PR-5873' 1.5492024491942589]
 ['PR-5874' 1.1433655135227045]
 ['PR-5875' 2.024925367391214]]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>